In [1]:
#importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np

In [10]:
!pip install selenium

In [59]:
#Kerala
#open the browser

driver=webdriver.Chrome()

#load the webpage

driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()
#implicit wait
wait = WebDriverWait(driver, 20)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
import numpy as np
def Kerala_link_route(path):   
    LINKS_KERALA = []
    ROUTE_KERALA = []

    # Retrieve the route links and route names
    for i in range(1, 4):  # Adjust the range based on the actual number of pages, e.g., 4
        paths = driver.find_elements(By.XPATH, path)
        
        # Retrieve links
        for links in paths:
            d = links.get_attribute("href")
            LINKS_KERALA.append(d)
        
        # Retrieve names of the routes
        for route in paths:
            ROUTE_KERALA.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            
            # Check if next button for the next page exists
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()={i+1}]'
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable(next_button))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

        except (NoSuchElementException, ElementClickInterceptedException) as e:
            print(f"Encountered an issue at step {i}: {e}")
            break
        
    return LINKS_KERALA, ROUTE_KERALA

# Calling the function to get links and routes
LINKS_KERALA, ROUTE_KERALA = Kerala_link_route("//a[@class='route']")

No more pages to paginate at step 2


In [60]:
df_k=pd.DataFrame({"Route_name":ROUTE_KERALA,"Route_link":LINKS_KERALA})

In [61]:
df_k

,Route_name,Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Bangalore to Kannur,https://www.redbus.in/bus-tickets/bangalore-to...
5,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
6,Kannur to Bangalore,https://www.redbus.in/bus-tickets/kannur-to-ba...
7,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
9,Bangalore to Kalpetta (kerala),https://www.redbus.in/bus-tickets/bangalore-to...


In [62]:
df = df_k

In [63]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [64]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv

In [65]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,09:00,10h 00m,INR 599,39 Seats available,4.3\n350,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,06:10,08h 10m,INR 599,29 Seats available,4.3\n290,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
2,NS Transports,Non A/C Seater / Sleeper (2+1),21:45,06:10,08h 25m,601,26 Seats available,4.1\n284,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
3,DLT BUS,A/C Sleeper (2+1),22:30,07:30,09h 00m,INR 900,17 Seats available,4.1\n315,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
4,P K Travels,A/C Seater / Sleeper (2+1),21:30,06:00,08h 30m,INR 750,27 Seats available,4.6\n15,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...,...
125,,,,,,,,,https://www.redbus.in/bus-tickets/kottayam-to-...,Kottayam to Kozhikode
126,,,,,,,,,https://www.redbus.in/bus-tickets/kottayam-to-...,Kottayam to Kozhikode
127,,,,,,,,,https://www.redbus.in/bus-tickets/kottayam-to-...,Kottayam to Kozhikode
128,,,,,,,,,https://www.redbus.in/bus-tickets/kottayam-to-...,Kottayam to Kozhikode


In [66]:
df1.to_csv("kerala.csv",index=False)

In [11]:
#open the browser
# 2Andhra
driver=webdriver.Chrome()

#load the webpage
driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()

In [15]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
import numpy as np

#implicit wait
wait = WebDriverWait(driver, 20)

def Andhra_link_route(path):   
    LINKS_Andhra= []
    ROUTE_Andhra = []

    # Retrieve the route links and route names
    for i in range(1, 5):  # Adjust the range based on the actual number of pages, e.g., 4
        paths = driver.find_elements(By.XPATH, path)
        
        # Retrieve links
        for links in paths:
            d = links.get_attribute("href")
            LINKS_Andhra.append(d)
        
        # Retrieve names of the routes
        for route in paths:
            ROUTE_Andhra.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            
            # Check if next button for the next page exists
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()={i+1}]'
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable(next_button))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

        except (NoSuchElementException, ElementClickInterceptedException) as e:
            print(f"Encountered an issue at step {i}: {e}")
            break
        
    return LINKS_Andhra,ROUTE_Andhra

# Calling the function to get links and routes
LINKS_Andhra, ROUTE_Andhra = Andhra_link_route("//a[@class='route']")


In [16]:
df_A=pd.DataFrame({"Route_name":ROUTE_Andhra,"Route_link":LINKS_Andhra})

In [17]:
df_A

,Route_name,Route_link
0,Hyderabad to Macherla (andhra pradesh),https://www.redbus.in/bus-tickets/hyderabad-to...
1,Hyderabad to Rajahmundry,https://www.redbus.in/bus-tickets/hyderabad-to...
2,Chilakaluripet to Hyderabad,https://www.redbus.in/bus-tickets/chilakalurip...
3,Rajahmundry to Hyderabad,https://www.redbus.in/bus-tickets/rajahmundry-...
4,Kurnool to Bangalore,https://www.redbus.in/bus-tickets/kurnool-to-b...
5,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Bangalore to Anantapur (andhra pradesh),https://www.redbus.in/bus-tickets/bangalore-to...
7,Bangalore to Chittoor (Andhra Pradesh),https://www.redbus.in/bus-tickets/bangalore-to...
8,Anantapur (andhra pradesh) to Bangalore,https://www.redbus.in/bus-tickets/ananthapur-t...
9,Hyderabad to Kurnool,https://www.redbus.in/bus-tickets/hyderabad-to...


In [19]:
df = df_A

In [20]:
#retrive the bus details
driver_A = webdriver.Chrome()
Bus_names_A = []
Bus_types_A = []
Start_Time_A = []
End_Time_A = []
Ratings_A= []
Total_Duration_A = []
Prices_A = []
Seats_Available_A = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_A.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_A.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_A.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_A.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_A).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_A.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_A.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_A.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_A.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_A.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_A.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_A.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_A.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_A.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_A.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_A.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_A.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_A.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_A.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_A.append(ratings.text)
    for price_elem in price:
        Prices_A.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_A.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [22]:

#from list to convert data frame
data = {
    'Bus_name': Bus_names_A,
    'Bus_type': Bus_types_A,
    'Start_time': Start_Time_A,
    'End_time': End_Time_A,
    'Total_duration': Total_Duration_A,
    'Price': Prices_A,
    "Seats_Available":Seats_Available_A,
    "Ratings":Ratings_A,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [23]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,APSRTC - 4403,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",12:15,16:50,04h 35m,INR 313,32 Seats available,4.2\n21,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Macherla (andhra pradesh)
1,APSRTC - 36403,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",13:00,17:15,04h 15m,INR 302,33 Seats available,4.3\n16,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Macherla (andhra pradesh)
2,APSRTC - 4399,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",13:45,18:00,04h 15m,INR 302,34 Seats available,2.1\n8,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Macherla (andhra pradesh)
3,APSRTC - 4405,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",14:15,18:30,04h 15m,INR 302,34 Seats available,4.1\n13,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Macherla (andhra pradesh)
4,APSRTC - 36409,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",15:00,19:15,04h 15m,INR 302,33 Seats available,3.9\n26,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Macherla (andhra pradesh)
...,...,...,...,...,...,...,...,...,...,...
320,NueGo,Electric A/C Seater (2+2),21:45,05:30,07h 45m,427,36 Seats available,4.1\n109,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Visakhapatnam
321,Yolo Bus,A/C Sleeper (2+1),22:20,06:33,08h 13m,INR 500,24 Seats available,4.6\n254,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Visakhapatnam
322,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:45,07:10,07h 25m,INR 464,32 Seats available,4.5\n388,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Visakhapatnam
323,NueGo,Electric A/C Seater/Sleeper (2+1),20:00,03:45,07h 45m,427,30 Seats available,4.2\n310,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Visakhapatnam


In [24]:
df1.to_csv("Andhra.csv",index=False)

In [57]:
#open the browser

driver=webdriver.Chrome()

#load the webpage
driver.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()

In [48]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
import numpy as np

#implicit wait
wait = WebDriverWait(driver, 20)

def Telangana_link_route(path):   
    LINKS_Telangana = []
    ROUTE_Telangana= []

    # Retrieve the route links and route names
    for i in range(1, 4):  # Adjust the range based on the actual number of pages, e.g., 4
        paths = driver.find_elements(By.XPATH, path)
        
        # Retrieve links
        for links in paths:
            d = links.get_attribute("href")
            LINKS_Telangana.append(d)
        
        # Retrieve names of the routes
        for route in paths:
            ROUTE_Telangana.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            
            # Check if next button for the next page exists
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()={i+1}]'
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable(next_button))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

        except (NoSuchElementException, ElementClickInterceptedException) as e:
            print(f"Encountered an issue at step {i}: {e}")
            break
        
    return LINKS_Telangana, ROUTE_Telangana

# Calling the function to get links and routes
LINKS_Telangana, ROUTE_Telangana = Telangana_link_route("//a[@class='route']")


No more pages to paginate at step 3


In [49]:
df_T=pd.DataFrame({"Route_name":ROUTE_Telangana,"Route_link":LINKS_Telangana})

In [50]:
df_T

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...


In [51]:
df=df_T

In [52]:
#retrive the bus details
driver_T = webdriver.Chrome()
Bus_names_T = []
Bus_types_T = []
Start_Time_T = []
End_Time_T = []
Ratings_T = []
Total_Duration_T = []
Prices_T = []
Seats_Available_T = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_T.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_T.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_T.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_T.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_T).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_T.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_T.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_T.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_T.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_T.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_T.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_T.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_T.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_T.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_T.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_T.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_T.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_T.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_T.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_T.append(ratings.text)
    for price_elem in price:
        Prices_T.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_T.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [53]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_T,
    'Bus_type': Bus_types_T,
    'Start_time': Start_Time_T,
    'End_time': End_Time_T,
    'Total_duration': Total_Duration_T,
    'Price': Prices_T,
    "Seats_Available":Seats_Available_T,
    "Ratings":Ratings_T,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv

In [44]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,TGSRTC - 4457,Super Luxury (Non AC Seater 2+2 Push Back),15:00,03:50,12h 50m,INR 764,27 Seats available,3.6\n12,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati
1,TGSRTC - 1711,Super Luxury (Non AC Seater 2+2 Push Back),15:00,03:30,12h 30m,INR 732,17 Seats available,3.6\n28,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati
2,TGSRTC - 7452,Super Luxury (Non AC Seater 2+2 Push Back),15:45,04:30,12h 45m,INR 732,24 Seats available,2.5\n11,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati
3,TGSRTC - 2474,Super Luxury (Non AC Seater 2+2 Push Back),16:30,04:00,11h 30m,INR 732,22 Seats available,3.1\n35,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati
4,TGSRTC - 9595,Lahari A/C sleeper,16:30,05:40,13h 10m,INR 1114,17 Seats available,4.0\n52,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati
...,...,...,...,...,...,...,...,...,...,...
271,KMBT Travels,NON A/C Hi-Tech Push Back (2+2),22:00,05:00,07h 00m,451,30 Seats available,3.4\n37,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Addanki
272,Sri Krishna Travels,A/C Semi Sleeper / Sleeper (2+1),21:45,04:30,06h 45m,INR 700,39 Seats available,3.2\n38,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Addanki
273,Orange Tours And Travels,NON A/C Seater/ Sleeper (2+1),22:30,06:00,07h 30m,INR 400,30 Seats available,3.6\n120,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Addanki
274,Mozo travels,A/C Sleeper (2+1),22:00,04:00,06h 00m,INR 800,19 Seats available,4.9\n6,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Addanki


In [55]:
df1.to_csv("Telangana.csv",index=False)

In [3]:
#4KTCL
#open the browser
driver=webdriver.Chrome()
#load the webpage
driver.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile")
time.sleep(3)
driver.maximize_window()


In [4]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
import numpy as np

#implicit wait
wait = WebDriverWait(driver, 20)

def Kadamba_link_route(path):   
    LINKS_Kadamba = []
    ROUTE_Kadamba= []

    # Retrieve the route links and route names
    for i in range(1, 5):  # Adjust the range based on the actual number of pages, e.g., 4
        paths = driver.find_elements(By.XPATH, path)
        
        # Retrieve links
        for links in paths:
            d = links.get_attribute("href")
            LINKS_Kadamba.append(d)
        
        # Retrieve names of the routes
        for route in paths:
            ROUTE_Kadamba.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            
            # Check if next button for the next page exists
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()={i+1}]'
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable(next_button))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

        except (NoSuchElementException, ElementClickInterceptedException) as e:
            print(f"Encountered an issue at step {i}: {e}")
            break
        
    return LINKS_Kadamba, ROUTE_Kadamba

# Calling the function to get links and routes
LINKS_Kadamba, ROUTE_Kadamba = Kadamba_link_route("//a[@class='route']")



No more pages to paginate at step 4


In [5]:
df_KG=pd.DataFrame({"Route_names":ROUTE_Kadamba,"Route_links":LINKS_Kadamba})

In [6]:
df_KG

,Route_names,Route_links
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,Goa to Pune,https://www.redbus.in/bus-tickets/goa-to-pune
2,Mumbai to Goa,https://www.redbus.in/bus-tickets/mumbai-to-goa
3,Bangalore to Goa,https://www.redbus.in/bus-tickets/bangalore-to...
4,Goa to Bangalore,https://www.redbus.in/bus-tickets/goa-to-banga...
5,Goa to Mumbai,https://www.redbus.in/bus-tickets/goa-to-mumbai
6,Pandharpur to Goa,https://www.redbus.in/bus-tickets/pandharpur-t...
7,Goa to Pandharpur,https://www.redbus.in/bus-tickets/goa-to-pandh...
8,Solapur to Goa,https://www.redbus.in/bus-tickets/solapur-to-goa
9,Calangute (goa) to Goa Airport,https://www.redbus.in/bus-tickets/calangute-go...


In [7]:
df=df_KG

In [8]:
#retrive the bus details
driver_KG = webdriver.Chrome()
Bus_names_KG = []
Bus_types_KG = []
Start_Time_KG = []
End_Time_KG = []
Ratings_KG = []
Total_Duration_KG = []
Prices_KG = []
Seats_Available_KG = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_links"]
    routes=r["Route_names"]

# Loop through each link
    driver_KG.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_KG.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_KG.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_KG.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_KG).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_KG.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_KG.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_KG.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_KG.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_KG.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_KG.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_KG.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_KG.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_KG.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_KG.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_KG.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_KG.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_KG.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_KG.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_KG.append(ratings.text)
    for price_elem in price:
        Prices_KG.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_KG.append(seats_elem.text)
        
print("Successfully Completed")
  

Successfully Completed


In [9]:
# from list to convert data frame
import pandas as pd
data = {
    'Bus_name': Bus_names_KG,
    'Bus_type': Bus_types_KG,
    'Start_time': Start_Time_KG,
    'End_time': End_Time_KG,
    'Total_duration': Total_Duration_KG,
    'Price': Prices_KG,
    "Seats_Available":Seats_Available_KG,
    "Ratings":Ratings_KG,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [10]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name


In [67]:
df1.to_csv("Goa.csv",index=False)

In [68]:
#5Rajasthan
#open the browser

driver=webdriver.Chrome()

#load the webpage

driver.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()

In [69]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
import numpy as np

#implicit wait
wait = WebDriverWait(driver, 20)

def Rajasthan_link_route(path):   
    LINKS_Rajasthan = []
    ROUTE_Rajasthan = []

    # Retrieve the route links and route names
    for i in range(1, 3):  # Adjust the range based on the actual number of pages, e.g., 4
        paths = driver.find_elements(By.XPATH, path)
        
        # Retrieve links
        for links in paths:
            d = links.get_attribute("href")
            LINKS_Rajasthan.append(d)
        
        # Retrieve names of the routes
        for route in paths:
            ROUTE_Rajasthan.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            
            # Check if next button for the next page exists
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()={i+1}]'
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable(next_button))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

        except (NoSuchElementException, ElementClickInterceptedException) as e:
            print(f"Encountered an issue at step {i}: {e}")
            break
        
    return LINKS_Rajasthan,ROUTE_Rajasthan

# Calling the function to get links and routes
LINKS_Rajasthan, ROUTE_Rajasthan = Rajasthan_link_route(("//a[@class='route']"))


No more pages to paginate at step 2


In [70]:
df_R=pd.DataFrame({"Route_names":ROUTE_Rajasthan,"Route_links":LINKS_Rajasthan})

In [71]:
df_R

,Route_names,Route_links
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
2,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
3,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
4,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
7,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...
8,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
9,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...


In [72]:
df=df_R

In [73]:
#retrive the bus details
driver_R= webdriver.Chrome()
Bus_names_R = []
Bus_types_R = []
Start_Time_R = []
End_Time_R = []
Ratings_R = []
Total_Duration_R = []
Prices_R = []
Seats_Available_R = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_links"]
    routes=r["Route_names"]

# Loop through each link
    driver_R.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_R.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_R.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_R.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_R).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_R.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_R.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_R.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_R.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_R.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_R.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_R.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_R.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_R.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_R.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_R.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_R.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_R.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_R.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_R.append(ratings.text)
    for price_elem in price:
        Prices_R.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_R.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [75]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_R,
    'Bus_type': Bus_types_R,
    'Start_time': Start_Time_R,
    'End_time': End_Time_R,
    'Total_duration': Total_Duration_R,
    'Price': Prices_R,
    "Seats_Available":Seats_Available_R,
    "Ratings":Ratings_R,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv

In [76]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,RSRTC - 174534,Express Non AC Seater 2+3,05:45,12:00,06h 15m,INR 299,42 Seats available,4.2,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
1,RSRTC - 158798,Super Luxury Volvo AC Seater Pushback 2+2,06:01,11:00,04h 59m,INR 602,12 Seats available,4.3\n144,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
2,RSRTC - 146395,Express Non AC Seater 2+3,06:45,14:00,07h 15m,INR 299,42 Seats available,4.2,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
3,RSRTC - 194255,Express Non AC Seater 2+3,07:45,14:20,06h 35m,INR 299,46 Seats available,4.2,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
4,RSRTC - 174533,Express Non AC Seater 2+3,08:45,16:00,07h 15m,INR 299,42 Seats available,4.2,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
...,...,...,...,...,...,...,...,...,...,...
132,Jakhar Travels,A/C Seater / Sleeper (2+1),19:30,23:55,04h 25m,INR 662,29 Seats available,3.8\n40,https://www.redbus.in/bus-tickets/jaipur-to-ko...,Jaipur (Rajasthan) to Kota(Rajasthan)
133,Babu Travels,NON A/C Seater/ Sleeper (2+1),22:15,04:45,06h 30m,INR 400,27 Seats available,3.0\n86,https://www.redbus.in/bus-tickets/jaipur-to-ko...,Jaipur (Rajasthan) to Kota(Rajasthan)
134,Jakhar Travels,NON A/C Seater/ Sleeper (2+1),21:15,03:00,05h 45m,INR 250,28 Seats available,3.8\n37,https://www.redbus.in/bus-tickets/jaipur-to-ko...,Jaipur (Rajasthan) to Kota(Rajasthan)
135,Raj travels Jhunjhunu,A/C Seater / Sleeper (2+1),23:30,05:00,05h 30m,380,25 Seats available,3.0\n52,https://www.redbus.in/bus-tickets/jaipur-to-ko...,Jaipur (Rajasthan) to Kota(Rajasthan)


In [77]:
df1.to_csv("Rajsthan.csv",index=False)

In [78]:

#6South Bengal
#open the browser

driver=webdriver.Chrome()

#load the webpage

driver.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc")

time.sleep(3)

driver.maximize_window()

In [79]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
import numpy as np

#implicit wait
wait = WebDriverWait(driver, 20)

def southbengal_link_route(path):   
    LINKS_southbengal = []
    ROUTE_southbengal = []

    # Retrieve the route links and route names
    for i in range(1, 6):  # Adjust the range based on the actual number of pages, e.g., 4
        paths = driver.find_elements(By.XPATH, path)
        
        # Retrieve links
        for links in paths:
            d = links.get_attribute("href")
            LINKS_southbengal.append(d)
        
        # Retrieve names of the routes
        for route in paths:
            ROUTE_southbengal.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            
            # Check if next button for the next page exists
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()={i+1}]'
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable(next_button))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

        except (NoSuchElementException, ElementClickInterceptedException) as e:
            print(f"Encountered an issue at step {i}: {e}")
            break
        
    return LINKS_southbengal, ROUTE_southbengal

# Calling the function to get links and routes
LINKS_southbengal, ROUTE_southbengal = southbengal_link_route("//a[@class='route']")


No more pages to paginate at step 5


In [80]:
df_s=pd.DataFrame({"Route_name":ROUTE_southbengal,"Route_link":LINKS_southbengal})

In [81]:
df_s

,Route_name,Route_link
0,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...
1,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...
2,Haldia to Calcutta,https://www.redbus.in/bus-tickets/haldia-to-ko...
3,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
4,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
5,Kolkata to Arambagh (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
6,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
7,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
8,Digha to Calcutta,https://www.redbus.in/bus-tickets/digha-to-kol...
9,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...


In [82]:
df=df_s

In [84]:
#retrive the bus details
driver_s = webdriver.Chrome()
Bus_names_s = []
Bus_types_s = []
Start_Time_s = []
End_Time_s = []
Ratings_s = []
Total_Duration_s = []
Prices_s = []
Seats_Available_s = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_s.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_s.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_s.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_s.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_s).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_s.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_s.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_s.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_s.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_s.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_s.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_s.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_s.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_s.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_s.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_s.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_s.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_s.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_s.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_s.append(ratings.text)
    for price_elem in price:
        Prices_s.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_s.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [85]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_s,
    'Bus_type': Bus_types_s,
    'Start_time': Start_Time_s,
    'End_time': End_Time_s,
    'Total_duration': Total_Duration_s,
    'Price': Prices_s,
    "Seats_Available":Seats_Available_s,
    "Ratings":Ratings_s,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv

In [87]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Pammi Travels,Non A/C Seater / Sleeper (2+1),02:00,06:00,04h 00m,INR 650,4 Seats available,3.7,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
1,Royal Cruiser,Scania Multi-Axle AC Semi Sleeper (2+2),02:00,06:00,04h 00m,509,25 Seats available,4.0\n88,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
2,Express Line (Karunamoyee),Volvo 9600 Multi Axle Semi-Sleeper (2+2),06:10,09:20,03h 10m,INR 486,30 Seats available,4.5\n179,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
3,Express Line (Karunamoyee),Volvo A/C Seater (2+2),06:40,10:45,04h 05m,INR 486,42 Seats available,4.5,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
4,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),06:40,10:00,03h 20m,INR 486,28 Seats available,4.4\n332,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
...,...,...,...,...,...,...,...,...,...,...
202,,,,,,,,,https://www.redbus.in/bus-tickets/barasat-west...,Barasat (West Bengal) to Digha
203,,,,,,,,,https://www.redbus.in/bus-tickets/barasat-west...,Barasat (West Bengal) to Digha
204,,,,,,,,,https://www.redbus.in/bus-tickets/barasat-west...,Barasat (West Bengal) to Digha
205,,,,,,,,,https://www.redbus.in/bus-tickets/barasat-west...,Barasat (West Bengal) to Digha


In [88]:
df1.to_csv("southbengal.csv",index=False)

In [89]:
#7Haryana
#open the browser

driver=webdriver.Chrome()

#load the webpage

driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()

In [90]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
import numpy as np

#implicit wait
wait = WebDriverWait(driver, 20)

def Haryana_link_route(path):   
    LINKS_Haryana = []
    ROUTE_Haryana= []

    # Retrieve the route links and route names
    for i in range(1, 5):  # Adjust the range based on the actual number of pages, e.g., 4
        paths = driver.find_elements(By.XPATH, path)
        
        # Retrieve links
        for links in paths:
            d = links.get_attribute("href")
            LINKS_Haryana.append(d)
        
        # Retrieve names of the routes
        for route in paths:
            ROUTE_Haryana.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            
            # Check if next button for the next page exists
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()={i+1}]'
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable(next_button))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

        except (NoSuchElementException, ElementClickInterceptedException) as e:
            print(f"Encountered an issue at step {i}: {e}")
            break
        
    return LINKS_Haryana, ROUTE_Haryana

# Calling the function to get links and routes
LINKS_Haryana, ROUTE_Haryana = Haryana_link_route("//a[@class='route']")


No more pages to paginate at step 4


In [91]:
df_h=pd.DataFrame({"Route_name":ROUTE_Haryana,"Route_link":LINKS_Haryana})

In [92]:
df_h

,Route_name,Route_link
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
2,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
3,Shimla to Delhi,https://www.redbus.in/bus-tickets/shimla-to-delhi
4,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
5,Hamirpur (Himachal Pradesh) to Delhi,https://www.redbus.in/bus-tickets/hamirpur-him...
6,Chamba (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/chamba-himac...
7,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...
8,Chandigarh to Dharamshala (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
9,Delhi to Chamba (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-cha...


In [93]:
df=df_h

In [94]:
#retrive the bus details
driver_h = webdriver.Chrome()
Bus_names_h = []
Bus_types_h = []
Start_Time_h = []
End_Time_h = []
Ratings_h = []
Total_Duration_h = []
Prices_h = []
Seats_Available_h = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_h.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_h.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_h.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_h.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_h).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_h.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_h.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_h.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_h.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_h.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_h.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_h.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_h.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_h.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_h.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_h.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_h.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_h.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_h.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_h.append(ratings.text)
    for price_elem in price:
        Prices_h.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_h.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [95]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_h,
    'Bus_type': Bus_types_h,
    'Start_time': Start_Time_h,
    'End_time': End_Time_h,
    'Total_duration': Total_Duration_h,
    'Price': Prices_h,
    "Seats_Available":Seats_Available_h,
    "Ratings":Ratings_h,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [96]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,HRTC - 6,Himsuta AC Seater Volvo/Scania 2+2,06:45,16:10,09h 25m,INR 912,22 Seats available,4.4\n58,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
1,HRTC - 592,A/C Executive (2+3),08:05,18:10,10h 05m,INR 632,41 Seats available,2.3\n12,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
2,HRTC - 129,Ordinary 3+2 Non AC Seater,08:50,18:40,09h 50m,INR 512,37 Seats available,2.6\n4,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
3,HRTC - 7,Himsuta AC Seater Volvo/Scania 2+2,09:25,18:30,09h 05m,INR 912,33 Seats available,4.6\n80,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
4,HRTC - 657,Ordinary 3+2 Non AC Seater,11:00,22:00,11h 00m,INR 512,33 Seats available,2.7\n8,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
...,...,...,...,...,...,...,...,...,...,...
181,,,,,,,,,https://www.redbus.in/bus-tickets/bilaspur-him...,Bilaspur (Himachal Pradesh) to Delhi
182,,,,,,,,,https://www.redbus.in/bus-tickets/bilaspur-him...,Bilaspur (Himachal Pradesh) to Delhi
183,,,,,,,,,https://www.redbus.in/bus-tickets/bilaspur-him...,Bilaspur (Himachal Pradesh) to Delhi
184,,,,,,,,,https://www.redbus.in/bus-tickets/bilaspur-him...,Bilaspur (Himachal Pradesh) to Delhi


In [97]:
df1.to_csv("Haryana.csv",index=False)

In [48]:
#8 Assam

#open the browser

driver=webdriver.Chrome()

#load the webpage

driver.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()

In [49]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
import numpy as np
#implicit wait
wait = WebDriverWait(driver, 20)

def Assam_link_route(path):   
    LINKS_Assam = []
    ROUTE_Assam = []

    # Retrieve the route links and route names
    for i in range(1, 5):  # Adjust the range based on the actual number of pages, e.g., 4
        paths = driver.find_elements(By.XPATH, path)
        
        # Retrieve links
        for links in paths:
            d = links.get_attribute("href")
            LINKS_Assam.append(d)
        
        # Retrieve names of the routes
        for route in paths:
            ROUTE_Assam.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            
            # Check if next button for the next page exists
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()={i+1}]'
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable(next_button))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

        except (NoSuchElementException, ElementClickInterceptedException) as e:
            print(f"Encountered an issue at step {i}: {e}")
            break
        
    return LINKS_Assam, ROUTE_Assam

# Calling the function to get links and routes
LINKS_Assam, ROUTE_Assam = Assam_link_route("//a[@class='route']")



In [50]:
df_As=pd.DataFrame({"Route_name":ROUTE_Assam,"Route_link":LINKS_Assam})

In [51]:
df_As

,Route_name,Route_link
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
2,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...
3,Nagaon (Assam) to Guwahati,https://www.redbus.in/bus-tickets/nagaon-to-gu...
4,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
5,Jorhat to North Lakhimpur,https://www.redbus.in/bus-tickets/jorhat-to-no...
6,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...
7,Jorhat to Dibrugarh,https://www.redbus.in/bus-tickets/jorhat-to-di...
8,North Lakhimpur to Jorhat,https://www.redbus.in/bus-tickets/north-lakhim...
9,North Lakhimpur to Sibsagar,https://www.redbus.in/bus-tickets/north-lakhim...


In [52]:
df=df_As

In [53]:
#retrive the bus details
driver_As = webdriver.Chrome()
Bus_names_As = []
Bus_types_As = []
Start_Time_As = []
End_Time_As = []
Ratings_As= []
Total_Duration_As = []
Prices_As = []
Seats_Available_As = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_As.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_As.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_As.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_As.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_As).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_As.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_As.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_As.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_As.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_As.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_As.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_As.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_As.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_As.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_As.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_As.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_As.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_As.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_As.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_As.append(ratings.text)
    for price_elem in price:
        Prices_As.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_As.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [54]:
# from list to convert data frame
import pandas as pd
data = {
    'Bus_name': Bus_names_As,
    'Bus_type': Bus_types_As,
    'Start_time': Start_Time_As,
    'End_time': End_Time_As,
    'Total_duration': Total_Duration_As,
    'Price': Prices_As,
    "Seats_Available":Seats_Available_As,
    "Ratings":Ratings_As,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv

In [55]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Assam State Transport Corporation (ASTC) - 157959,Bharat Benz A/C Seater (2+2),14:00,18:30,04h 30m,INR 298,24 Seats available,4.2\n206,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
1,Assam State Transport Corporation (ASTC) - 154930,Volvo AC Seater 2+2,15:45,20:00,04h 15m,INR 298,21 Seats available,4.0\n54,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
2,Assam State Transport Corporation (ASTC) - 161055,Volvo AC Seater 2+2,16:15,20:30,04h 15m,INR 298,19 Seats available,4.4\n81,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
3,Chartered Bus - ASTC,A/C Seater Push Back (2+2),11:30,16:15,04h 45m,INR 334.68,14 Seats available,4.1\n111,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
4,Chartered Bus - ASTC,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),13:00,17:00,04h 00m,INR 389.13,30 Seats available,4.7\n481,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
5,Chartered Bus - ASTC,Volvo AC Seater Pushback 2+2,14:30,18:30,04h 00m,INR 432.06,31 Seats available,4.6\n118,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
6,SHIVAM TRAVELS,Bharat Benz A/C Seater (2+1),15:00,19:15,04h 15m,324,32 Seats available,4.1\n198,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
7,Chartered Bus - ASTC,Volvo AC Seater Pushback 2+2,15:30,19:15,03h 45m,INR 389.13,32 Seats available,4.6\n486,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
8,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),22:15,03:45,05h 30m,450,21 Seats available,3.6\n27,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
9,Himalayan Travels,NON AC Seater / Sleeper 2+1,23:40,04:00,04h 20m,INR 500,31 Seats available,1.7\n17,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati


In [56]:
df1.to_csv("Assam.csv",index=False)

In [107]:
#9Uttar Pradesh

#open the browser

driver=webdriver.Chrome()

#load the webpage

driver.get("https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()

In [108]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
import numpy as np

#implicit wait
wait = WebDriverWait(driver, 20)

def UttarPradesh_link_route(path):   
    LINKS_UttarPradesh = []
    ROUTE_UttarPradesh = []

    # Retrieve the route links and route names
    for i in range(1, 6):  # Adjust the range based on the actual number of pages, e.g., 4
        paths = driver.find_elements(By.XPATH, path)
        
        # Retrieve links
        for links in paths:
            d = links.get_attribute("href")
            LINKS_UttarPradesh.append(d)
        
        # Retrieve names of the routes
        for route in paths:
            ROUTE_UttarPradesh.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            
            # Check if next button for the next page exists
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()={i+1}]'
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable(next_button))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

        except (NoSuchElementException, ElementClickInterceptedException) as e:
            print(f"Encountered an issue at step {i}: {e}")
            break
        
    return LINKS_UttarPradesh, ROUTE_UttarPradesh

# Calling the function to get links and routes
LINKS_UttarPradesh, ROUTE_UttarPradesh = UttarPradesh_link_route("//a[@class='route']")


No more pages to paginate at step 5


In [109]:
df_UP=pd.DataFrame({"Route_name":ROUTE_UttarPradesh,"Route_link":LINKS_UttarPradesh})

In [110]:
df_UP

,Route_name,Route_link
0,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...
1,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
2,Aligarh (uttar pradesh) to Delhi,https://www.redbus.in/bus-tickets/aligarh-utta...
3,Delhi to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/delhi-to-ali...
4,Lucknow to Allahabad,https://www.redbus.in/bus-tickets/lucknow-to-a...
5,Lucknow to Delhi,https://www.redbus.in/bus-tickets/lucknow-to-d...
6,Delhi to Farrukhabad (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-far...
7,Farrukhabad (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/farrukhabad-...
8,Badaun to Delhi,https://www.redbus.in/bus-tickets/badaun-to-delhi
9,Allahabad to Lucknow,https://www.redbus.in/bus-tickets/allahabad-to...


In [111]:
df=df_UP

In [112]:
#retrive the bus details
driver_UP = webdriver.Chrome()
Bus_names_UP = []
Bus_types_UP = []
Start_Time_UP = []
End_Time_UP = []
Ratings_UP = []
Total_Duration_UP = []
Prices_UP = []
Seats_Available_UP = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_UP.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_UP.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_UP.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_UP.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_UP).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_UP.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_UP.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_UP.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_UP.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_UP.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_UP.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_UP.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_UP.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_UP.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_UP.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_UP.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_UP.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_UP.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_UP.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_UP.append(ratings.text)
    for price_elem in price:
        Prices_UP.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_UP.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [113]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_UP,
    'Bus_type': Bus_types_UP,
    'Start_time': Start_Time_UP,
    'End_time': End_Time_UP,
    'Total_duration': Total_Duration_UP,
    'Price': Prices_UP,
    "Seats_Available":Seats_Available_UP,
    "Ratings":Ratings_UP,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv

In [114]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,UPSRTC - FKP0028,Janrath AC Seater 2+2,08:00,16:55,08h 55m,INR 710,34 Seats available,3.9\n4,https://www.redbus.in/bus-tickets/delhi-to-far...,Delhi to Farrukhabad (Uttar Pradesh)
1,UPSRTC - FKP0030,Janrath AC Seater 2+2,09:00,17:55,08h 55m,INR 710,36 Seats available,3.5\n4,https://www.redbus.in/bus-tickets/delhi-to-far...,Delhi to Farrukhabad (Uttar Pradesh)
2,UPSRTC - LNI0046,Ordinary Non AC Seater 2+3,09:30,18:35,09h 05m,INR 512,52 Seats available,3.3,https://www.redbus.in/bus-tickets/delhi-to-far...,Delhi to Farrukhabad (Uttar Pradesh)
3,UPSRTC - LNI0319,Ordinary Non AC Seater 2+3,10:00,16:30,06h 30m,INR 512,52 Seats available,3.3,https://www.redbus.in/bus-tickets/delhi-to-far...,Delhi to Farrukhabad (Uttar Pradesh)
4,UPSRTC - SBD0105,Janrath AC Seater 2+2,10:01,18:30,08h 29m,INR 710,36 Seats available,2.5\n17,https://www.redbus.in/bus-tickets/delhi-to-far...,Delhi to Farrukhabad (Uttar Pradesh)
...,...,...,...,...,...,...,...,...,...,...
121,UPSRTC - CBG0093,Pink Express AC Seater 2+2,19:16,02:45,07h 29m,INR 840,41 Seats available,2.6\n9,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
122,UPSRTC - RKD0057,Ordinary Non AC Seater 2+3,19:31,03:00,07h 29m,INR 517,52 Seats available,3.1\n4,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
123,UPSRTC - RKD0135,Janrath AC Seater 2+3,22:31,06:01,07h 30m,INR 652,48 Seats available,2.4\n39,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
124,Mahalaxmi Travels,Bharat Benz A/C Seater /Sleeper (2+1),17:00,01:45,08h 45m,INR 699,40 Seats available,3.6,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly


In [115]:
df1.to_csv("UttarPradesh.csv",index=False)

In [198]:
# 10 WestBengal
#open the browser

driver=webdriver.Chrome()

#load the webpage

driver.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()

In [199]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
import numpy as np

#implicit wait
wait = WebDriverWait(driver, 20)

def WestBengal_link_route(path):   
    LINKS_WestBengal = []
    ROUTE_WestBengal = []

    # Retrieve the route links and route names
    for i in range(1, 5):  # Adjust the range based on the actual number of pages, e.g., 4
        paths = driver.find_elements(By.XPATH, path)
        
        # Retrieve links
        for links in paths:
            d = links.get_attribute("href")
            LINKS_WestBengal.append(d)
        
        # Retrieve names of the routes
        for route in paths:
            ROUTE_WestBengal.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            
            # Check if next button for the next page exists
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()={i+1}]'
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable(next_button))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

        except (NoSuchElementException, ElementClickInterceptedException) as e:
            print(f"Encountered an issue at step {i}: {e}")
            break
        
    return LINKS_WestBengal, ROUTE_WestBengal

# Calling the function to get links and routes
LINKS_WestBengal, ROUTE_WestBengal = WestBengal_link_route("//a[@class='route']")


No more pages to paginate at step 4


In [200]:
df_WB=pd.DataFrame({"Route_name":ROUTE_WestBengal,"Route_link":LINKS_WestBengal})

In [201]:
df_WB

,Route_name,Route_link
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...
2,Digha to Calcutta,https://www.redbus.in/bus-tickets/digha-to-kol...
3,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
4,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...
5,Kolkata to Suri,https://www.redbus.in/bus-tickets/kolkata-to-suri
6,Barasat (West Bengal) to Midnapore,https://www.redbus.in/bus-tickets/barasat-west...
7,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
8,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...
9,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...


In [202]:
df=df_WB

In [203]:
#retrive the bus details
driver_WB = webdriver.Chrome()
Bus_names_WB = []
Bus_types_WB = []
Start_Time_WB = []
End_Time_WB = []
Ratings_WB = []
Total_Duration_WB = []
Prices_WB = []
Seats_Available_WB = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_WB.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_WB.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_WB.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_WB.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_WB).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_WB.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_WB.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_WB.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_WB.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_WB.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_WB.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_WB.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_WB.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_WB.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_WB.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_WB.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_WB.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_WB.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_WB.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_WB.append(ratings.text)
    for price_elem in price:
        Prices_WB.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_WB.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [204]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_WB,
    'Bus_type': Bus_types_WB,
    'Start_time': Start_Time_WB,
    'End_time': End_Time_WB,
    'Total_duration': Total_Duration_WB,
    'Price': Prices_WB,
    "Seats_Available":Seats_Available_WB,
    "Ratings":Ratings_WB,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv

In [205]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,WBTC (CTC) HABRA-DIGHA via Esplanade - 176|04:00,Non AC Seater (2+3),04:00,09:50,05h 50m,INR 165,41 Seats available,2.1\n7,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
1,WBTC (CTC) BARASAT-DIGHA via Bally - 38|05:40,Non AC Seater (2+3),05:40,10:10,04h 30m,INR 161,40 Seats available,3.2\n100,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
2,WBTC (CTC) BADURIA - DIGHA via Bally - 155|09:00,Non AC Seater (2+3),09:00,12:25,03h 25m,INR 161,42 Seats available,2.7\n17,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
3,WBTC (CTC) HABRA-DIGHA via Bally - 26|10:00,Non AC Seater (2+3),10:00,14:45,04h 45m,INR 161,48 Seats available,3.4\n47,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
4,"WBTC (CTC) BARASAT-DIGHA via Esplanade, Karuna...",Non AC Seater (2+3),10:15,15:35,05h 20m,INR 165,47 Seats available,3.0\n4,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
...,...,...,...,...,...,...,...,...,...,...
128,WBTC (CTC) HABRA-DIGHA via Bally - 26|03:45,Non AC Seater (2+3),03:45,07:05,03h 20m,INR 130,44 Seats available,4.2,https://www.redbus.in/bus-tickets/habra-to-heria,Habra to Heria
129,WBTC (CTC) HABRA-DIGHA via Bally - 26|04:15,Non AC Seater (2+3),04:15,07:35,03h 20m,INR 130,45 Seats available,4.5\n4,https://www.redbus.in/bus-tickets/habra-to-heria,Habra to Heria
130,WBTC (CTC) HABRA-DIGHA via Bally - 26|04:30,Non AC Seater (2+3),04:30,08:10,03h 40m,INR 130,42 Seats available,4.3\n5,https://www.redbus.in/bus-tickets/habra-to-heria,Habra to Heria
131,WBTC (CTC) HABRA-DIGHA via Bally - 26|05:15,Non AC Seater (2+3),05:15,08:35,03h 20m,INR 130,26 Seats available,3.8\n8,https://www.redbus.in/bus-tickets/habra-to-heria,Habra to Heria


In [206]:
df1.to_csv("WestBengal.csv",index=False)

In [3]:
import pandas as pd
import pymysql
#list of csv file paths
csv_files=["Kerala.csv","Andhra.csv","Telangana.csv","Goa.csv","Rajsthan.csv","Southbengal.csv","Haryana.csv",
           "Assam.csv","WestBengal.csv","UttarPradesh.csv",]
#Read each CSV file into
df_list=[pd.read_csv(file)for file in csv_files]
#concatenate all Dataframes in the list
combined_df=pd.concat(df_list,ignore_index=True)









In [4]:
combined_df

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,09:00,10h 00m,INR 599,39 Seats available,4.3\n350,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,06:10,08h 10m,INR 599,29 Seats available,4.3\n290,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
2,NS Transports,Non A/C Seater / Sleeper (2+1),21:45,06:10,08h 25m,601,26 Seats available,4.1\n284,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
3,DLT BUS,A/C Sleeper (2+1),22:30,07:30,09h 00m,INR 900,17 Seats available,4.1\n315,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
4,P K Travels,A/C Seater / Sleeper (2+1),21:30,06:00,08h 30m,INR 750,27 Seats available,4.6\n15,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...,...
1151,UPSRTC - CBG0093,Pink Express AC Seater 2+2,19:16,02:45,07h 29m,INR 840,41 Seats available,2.6\n9,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1152,UPSRTC - RKD0057,Ordinary Non AC Seater 2+3,19:31,03:00,07h 29m,INR 517,52 Seats available,3.1\n4,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1153,UPSRTC - RKD0135,Janrath AC Seater 2+3,22:31,06:01,07h 30m,INR 652,48 Seats available,2.4\n39,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1154,Mahalaxmi Travels,Bharat Benz A/C Seater /Sleeper (2+1),17:00,01:45,08h 45m,INR 699,40 Seats available,3.6,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly


In [5]:
combined_df.to_csv("RedBusRoute.csv",index=False)

In [6]:
#Extract digits from'seat Available'column(if exists)and handle NaN values
if 'Seats_Available' in combined_df.columns:
    combined_df['Seats_Available']=combined_df['Seats_Available'].fillna('').str.extract(r'(\d+)')

In [7]:
combined_df

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,09:00,10h 00m,INR 599,39,4.3\n350,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,06:10,08h 10m,INR 599,29,4.3\n290,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
2,NS Transports,Non A/C Seater / Sleeper (2+1),21:45,06:10,08h 25m,601,26,4.1\n284,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
3,DLT BUS,A/C Sleeper (2+1),22:30,07:30,09h 00m,INR 900,17,4.1\n315,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
4,P K Travels,A/C Seater / Sleeper (2+1),21:30,06:00,08h 30m,INR 750,27,4.6\n15,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...,...
1151,UPSRTC - CBG0093,Pink Express AC Seater 2+2,19:16,02:45,07h 29m,INR 840,41,2.6\n9,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1152,UPSRTC - RKD0057,Ordinary Non AC Seater 2+3,19:31,03:00,07h 29m,INR 517,52,3.1\n4,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1153,UPSRTC - RKD0135,Janrath AC Seater 2+3,22:31,06:01,07h 30m,INR 652,48,2.4\n39,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1154,Mahalaxmi Travels,Bharat Benz A/C Seater /Sleeper (2+1),17:00,01:45,08h 45m,INR 699,40,3.6,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly


In [8]:
#Replace'INR" in 'Price' column(if exists) and handle NAN values
import pandas as pd
import numpy as np

combined_df['Price'] = combined_df['Price'].str.replace('INR', '').str.replace(' ', '').astype(float)




In [9]:
combined_df

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,09:00,10h 00m,599.0,39,4.3\n350,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,06:10,08h 10m,599.0,29,4.3\n290,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
2,NS Transports,Non A/C Seater / Sleeper (2+1),21:45,06:10,08h 25m,601.0,26,4.1\n284,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
3,DLT BUS,A/C Sleeper (2+1),22:30,07:30,09h 00m,900.0,17,4.1\n315,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
4,P K Travels,A/C Seater / Sleeper (2+1),21:30,06:00,08h 30m,750.0,27,4.6\n15,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...,...
1151,UPSRTC - CBG0093,Pink Express AC Seater 2+2,19:16,02:45,07h 29m,840.0,41,2.6\n9,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1152,UPSRTC - RKD0057,Ordinary Non AC Seater 2+3,19:31,03:00,07h 29m,517.0,52,3.1\n4,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1153,UPSRTC - RKD0135,Janrath AC Seater 2+3,22:31,06:01,07h 30m,652.0,48,2.4\n39,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1154,Mahalaxmi Travels,Bharat Benz A/C Seater /Sleeper (2+1),17:00,01:45,08h 45m,699.0,40,3.6,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly


In [14]:
import pandas as pd
import numpy as np
combined_df['Ratings'] = combined_df['Ratings'].replace(r'[\n]', '', regex=True)
combined_df['Ratings'] = pd.to_numeric(combined_df['Ratings'], errors='coerce')

In [15]:
combined_df

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,09:00,10h 00m,599.0,39,4.3350,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,06:10,08h 10m,599.0,29,4.3290,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
2,NS Transports,Non A/C Seater / Sleeper (2+1),21:45,06:10,08h 25m,601.0,26,4.1284,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
3,DLT BUS,A/C Sleeper (2+1),22:30,07:30,09h 00m,900.0,17,4.1315,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
4,P K Travels,A/C Seater / Sleeper (2+1),21:30,06:00,08h 30m,750.0,27,4.6150,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...,...
1151,UPSRTC - CBG0093,Pink Express AC Seater 2+2,19:16,02:45,07h 29m,840.0,41,2.6900,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1152,UPSRTC - RKD0057,Ordinary Non AC Seater 2+3,19:31,03:00,07h 29m,517.0,52,3.1400,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1153,UPSRTC - RKD0135,Janrath AC Seater 2+3,22:31,06:01,07h 30m,652.0,48,2.4390,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1154,Mahalaxmi Travels,Bharat Benz A/C Seater /Sleeper (2+1),17:00,01:45,08h 45m,699.0,40,3.6000,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly


In [16]:
combined_df.to_csv('RedBusRoute.csv', index=False)

In [17]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('RedBusRoute.csv')

# Remove rows with any null values
df_cleaned = df.dropna()

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv('RedBusRoute.csv', index=False)

In [20]:
df_cleaned

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,09:00,10h 00m,599.0,39.0,4.3350,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,06:10,08h 10m,599.0,29.0,4.3290,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
2,NS Transports,Non A/C Seater / Sleeper (2+1),21:45,06:10,08h 25m,601.0,26.0,4.1284,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
3,DLT BUS,A/C Sleeper (2+1),22:30,07:30,09h 00m,900.0,17.0,4.1315,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
4,P K Travels,A/C Seater / Sleeper (2+1),21:30,06:00,08h 30m,750.0,27.0,4.6150,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...,...
1151,UPSRTC - CBG0093,Pink Express AC Seater 2+2,19:16,02:45,07h 29m,840.0,41.0,2.6900,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1152,UPSRTC - RKD0057,Ordinary Non AC Seater 2+3,19:31,03:00,07h 29m,517.0,52.0,3.1400,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1153,UPSRTC - RKD0135,Janrath AC Seater 2+3,22:31,06:01,07h 30m,652.0,48.0,2.4390,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly
1154,Mahalaxmi Travels,Bharat Benz A/C Seater /Sleeper (2+1),17:00,01:45,08h 45m,699.0,40.0,3.6000,https://www.redbus.in/bus-tickets/dehradun-to-...,Dehradun to Bareilly


In [21]:
import pymysql
import pandas as pd

# Database connection parameters
host = '127.0.0.1'
user = 'root'
password = '2910vvvi'
database_name = 'Redbus_data'

# Connect to the MySQL server
connection = pymysql.connect(host='127.0.0.1',user='root', password='2910vvvi')
cursor = connection.cursor()

# Create a new database
cursor.execute(f"CREATE DATABASE IF NOT EXISTS Redbus_data")
cursor.execute(f"USE Redbus_data")

# Create a new table
create_table_query = """
CREATE TABLE IF NOT EXISTS Bus_Route (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    Bus_name TEXT,
    Bus_type TEXT,
    Start_time TIME,
    End_time TIME,
    Total_duration TEXT,
    Price DECIMAL(10, 2),
    Seats_Available INT,
    Rating FLOAT,
    Route_name TEXT,
    Route_link TEXT
);
"""
cursor.execute(create_table_query)

# Load CSV into a DataFrame
df_cleaned = pd.read_csv('RedBusRoute.csv')

# Insert DataFrame into the MySQL table
for index, row in df.iterrows():
    cursor.execute('''
        INSERT INTO Bus_Route (Bus_name, Bus_type, Start_time, End_time, 
                               Total_duration, Price, Seats_Available, Rating, 
                               Route_name, Route_link)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ''', tuple(row))

# Commit the transaction
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


In [ ]:
import streamlit as st
import pandas as pd
import pymysql
import re  # Import regular expressions module for pattern matching
import requests  # Import requests for Lottie animation loading


heading_text = "<span style='color:#FF0000'>Redbus Ticket Booking</span>"
st.markdown(f"<h1>{heading_text}</h1>", unsafe_allow_html=True)



# Database credentials
user = 'root'
password = '2910vvvi'
host = '127.0.0.1'
database = 'redbus_data'

# Connect to MySQL database
conn = pymysql.connect(
    user=user,
    password=password,
    host=host,
    database=database
)


# Select RTC
rtc_options = [
    'RedbusRoute'
]
selected_rtc = st.selectbox('Select RTC:', rtc_options)

# Construct SQL query based on selected RTC
query = f"SELECT * FROM {selected_rtc}"

# Execute SQL query and fetch data into a DataFrame
try:
    df = pd.read_sql(query, conn)
except Exception as e:
    st.error(f"Error fetching data: {str(e)}")
    df = pd.DataFrame()  # Empty DataFrame in case of error

# Display RTC data table
st.subheader(f'Table for {selected_rtc}')
st.write(df)

# Filter options
selected_routes = st.multiselect('Select Bus Route(s):', df['Route_Name'].unique() if not df.empty else [])
selected_bus_types = st.multiselect('Select Bus Type(s):', df['Bus_Type'].unique() if not df.empty else [])

# Calculate min and max price range from the data
min_price = int(df['Price'].min()) if not df.empty else 0
max_price = int(df['Price'].max()) if not df.empty else 1000
price_range = st.slider('Price Range:', min_value=min_price, max_value=max_price, value=(min_price, max_price))

star_rating = st.slider('Star Rating:', min_value=1.0, max_value=5.0, step=0.1, value=(1.0, 5.0))
seats_available = st.slider('Seats Available:', min_value=0, max_value=100, value=(0, 100))

# Apply filters and fetch data based on user selections
filters = []

if selected_routes:
    route_conditions = " OR ".join([f"Route_Name = '{route}'" for route in selected_routes])
    filters.append(f"({route_conditions})")

if selected_bus_types:
    type_conditions = " OR ".join([f"Bus_Type = '{bus_type}'" for bus_type in selected_bus_types])
    filters.append(f"({type_conditions})")

filters.append(f"Price BETWEEN {price_range[0]} AND {price_range[1]}")
filters.append(f"Star_Rating BETWEEN {star_rating[0]} AND {star_rating[1]}")
filters.append(f"Seat_Availability BETWEEN {seats_available[0]} AND {seats_available[1]}")

# Construct SQL query with filters
if filters:
    filter_query = " AND ".join(filters)
    filtered_query = f"{query} WHERE {filter_query}"
else:
    filtered_query = query

# Execute filtered SQL query and fetch data into a DataFrame
try:
    filtered_df = pd.read_sql(filtered_query, conn)
    st.subheader('Filtered Results')
    st.write(filtered_df)
except Exception as e:
    st.error(f"Error fetching filtered data: {str(e)}")

# Close the database connection
conn.close()

